### AJUSTE HIPERPARAMETROS RED NEURONAL ARTIFICIAL COEFICIENTE DE ACTIVIDAD Y COEFICIENTE OSMOTICO.

### Cargar los datos ya limpiados y analizados en el Notebook analisis data y cargar librerias necesarias

In [4]:
!pip install -q -U keras-tuner

In [1]:
import numpy as np
import pandas as pd
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
datos=pd.read_csv("Data_clean_Completo4.csv")



### Preprocesamiento de datos

In [2]:

variables_categoricas = ["Cation", "Anion","T_Comportamiento"]
variables_numericas = ["Solute_Mol_Mass", "Num_Cations", "Num_Anions", "Temperature_C", "Solvent_Mol_Mass", "Molal"]
variables_a_predecir = ["Molal_Activity_Coefficient", "Osmotic_Coefficient"]
column_transformer = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), variables_categoricas),
        ("num", StandardScaler(), variables_numericas)
    ])

X = column_transformer.fit_transform(datos[variables_categoricas + variables_numericas])
y = datos[["Molal_Activity_Coefficient", "Osmotic_Coefficient", "Solvent_Activity"]].values


In [3]:
# División de datos en conjuntos de entrenamiento y prueba

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(datos, test_size=0.2, random_state=42, stratify=datos['Id_solucion'])
X_test = column_transformer.transform(test_data[variables_categoricas + variables_numericas])
y_test = test_data[variables_a_predecir].values
X_train = column_transformer.transform(train_data[variables_categoricas + variables_numericas])
y_train = train_data[variables_a_predecir].values
X_train=X_train.toarray()


### OPTIMIZACION DE HP ANN-MLP

In [11]:
from tensorflow import keras
from keras.layers import Activation
from kerastuner import HyperParameters
from keras.regularizers import l1_l2
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
def smish(x, beta=1):
    return x * K.tanh(K.softplus(beta * x))
Activation(smish)

def combined_loss(y_true, y_pred, alpha=0.08, weightA=0.6):
    mape_1 = K.mean(K.abs((y_true[:,0] - y_pred[:,0]) / K.clip(K.abs(y_true[:,0]), K.epsilon(), None))) * 100
    mape_2 = K.mean(K.abs((y_true[:,1] - y_pred[:,1]) / K.clip(K.abs(y_true[:,1]), K.epsilon(), None))) * 100
    #mape_3=  K.mean(K.abs((y_true[:,2] - y_pred[:,2]) / K.clip(K.abs(y_true[:,2]), K.epsilon(), None))) * 100
    mape = weightA*mape_1 +(1-weightA)*mape_2   #+(1-weightA-weightB)*mape_3
    mse_1 = K.mean(K.square(y_true[:,0] - y_pred[:,0]))
    mse_2 = K.mean(K.square(y_true[:,1] - y_pred[:,1]))
    #mse_3 = K.mean(K.square(y_true[:,2] - y_pred[:,2]))
    mse = weightA*mse_1 +(1-weightA)*mse_2#+(1-weightA-weightB)*mse_3
    return alpha * mape + (1 - alpha) * mse

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15)
# Definicion hiperparámetros y espacio de búsqueda
hp = HyperParameters()

num_hidden_layers = hp.Int('num_hidden_layers', 1, 3, default=1)
num_neurons = hp.Choice('num_neurons', values=[5, 15, 20], default=15)
optimizer = hp.Choice('optimizer', values=['adam', 'adamax'], default='adam')
learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001], default=0.01)
batch_size = hp.Choice('batch_size', values=[32, 64, 128, 256], default=128)
activation = hp.Choice('activation', values=['relu', 'swish', 'smish'], default='relu')
#alpha = hp.Float('alpha', 0, 1, default=0.08)
#weight_gamma_phi = hp.Float('weight_gamma_phi', 0, 1, default=0.6)

# Definicion modelo 
def build_model(hp):
    inputs = keras.Input(shape=(175,))
    x = layers.Dense(num_neurons, kernel_initializer='random_uniform', activation=activation, kernel_regularizer=l1_l2(l1=0.0001, l2=0))(inputs)
    for i in range(num_hidden_layers - 1):
        x = layers.Dense(num_neurons, kernel_initializer='random_uniform', activation=activation, kernel_regularizer=l1_l2(l1=0.0001, l2=0))(x)
    outputs = layers.Dense(2, kernel_initializer='random_uniform', activation=smish)(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=combined_loss, metrics="MAPE")
    return model

# Defificinion metodo de busqueda
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    hyperparameters=hp,
    directory='random_search',
    project_name='keras_tuner')

# Ejecucion búsqueda aleatoria
tuner.search(X_train, y_train, validation_split=0.1, epochs=100, batch_size=batch_size, callbacks=[callback])


Trial 10 Complete [00h 00m 29s]
val_loss: 2.0638184547424316

Best val_loss So Far: 1.6415352821350098
Total elapsed time: 00h 04m 14s
INFO:tensorflow:Oracle triggered exit


In [14]:
tuner.results_summary()

Results summary
Results in random_search/keras_tuner
Showing 10 best trials
Trial summary
Hyperparameters:
num_hidden_layers: 2
num_neurons: 20
optimizer: adamax
learning_rate: 0.1
batch_size: 256
activation: relu
alpha: 0.8711270956615105
weight_gamma_phi: 0.8421553974721291
Score: 1.6415352821350098
Trial summary
Hyperparameters:
num_hidden_layers: 1
num_neurons: 5
optimizer: adam
learning_rate: 0.01
batch_size: 32
activation: smish
alpha: 0.6177907485452416
weight_gamma_phi: 0.31783347911048154
Score: 1.6899281740188599
Trial summary
Hyperparameters:
num_hidden_layers: 2
num_neurons: 15
optimizer: adamax
learning_rate: 0.01
batch_size: 64
activation: relu
alpha: 0.29910974726671724
weight_gamma_phi: 0.974452342747445
Score: 1.7796812057495117
Trial summary
Hyperparameters:
num_hidden_layers: 1
num_neurons: 15
optimizer: adamax
learning_rate: 0.001
batch_size: 128
activation: swish
alpha: 0.8670475391920119
weight_gamma_phi: 0.62771193091902
Score: 1.809887409210205
Trial summary
Hyp